In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on='title') # merging two database on the basis of a common column

In [ ]:
movies.head(1)

In [ ]:
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum() #computing the sum of all null values in each colom

In [ ]:
movies.dropna(inplace=True) # remove rows containing any missing values (NaN or null) from the DataFrame 

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast #In your case, the data_string is a JSON-like string representing a list of dictionaries. 
# The ast.literal_eval() function will safely convert this string into a Python list of dictionaries:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['genres'][0]

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies['keywords'][0]

In [ ]:
movies['cast'][0]

In [ ]:
movies.head()

In [ ]:
def convert(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert)

In [ ]:
movies['cast'][0]

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'].apply(fetch_director)

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())# The split() method, when called on a string, splits the string into a list of substrings based on whitespace characters by default.

In [ ]:
movies['overview'][0]

In [ ]:
movies.head()

In [ ]:
# we have to remove space 'Johnny Depp' -> 'JohnnyDepp'

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
# The lambda function is replacing spaces with empty strings for each element in the list.

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
#stemming
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
# It appears that you have a function named stem that takes a text input, splits it into words, applies stemming to 
# each word using a stemmer (presumably ps), and then joins the stemmed words back into a single string separated by spaces.

In [ ]:
['dancing','dance']
ps.stem('dancing')

In [ ]:
new_df['tags'][0]

In [ ]:
#convert all in lower case

new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english') # "the," "and," "is," and other common English stop words are ignored during the transformation, and they don't appear in the resulting feature matrix.


In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
# we use cosine distance 
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title']== movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse= True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(i)

In [ ]:
new_df[new_df['title'] == 'Batman Begins'].index[0]

In [ ]:
list(enumerate(similarity[1]))

In [ ]:
sorted(list(enumerate(similarity[0])),reverse= True,key=lambda x:x[1])[1:6]

In [ ]:
recommend("Avatar")

In [ ]:
recommend("Batman Begins")

In [ ]:
new_df.iloc[65].title

In [ ]:
# import pickle

In [ ]:
## pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
# pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
# pickle.dump(similarity,open('similarity.pkl','wb'))